<a href="https://colab.research.google.com/github/Mahim05078/CSE6813--FedLearning-MLP-IDS-Systems/blob/master/IDS_using_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# So first we need to download our dataset...
# It can be found in from this url..
!git clone 'https://github.com/Mahim05078/NSL-KDD-Dataset.git'

Cloning into 'NSL-KDD-Dataset'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 15 (delta 6), reused 15 (delta 6), pack-reused 0
Unpacking objects: 100% (15/15), 5.13 MiB | 2.66 MiB/s, done.


In [ ]:
#import basic libraries
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn import decomposition, preprocessing, svm

In [ ]:
print(os.getcwd())

/content


In [ ]:
#load the necessary features from training testing and validation data
data = pd.read_csv('NSL-KDD-Dataset/input/nslkdd/KDDTrain+.txt',header = None, names=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent'
            ,'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root'
            ,'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login'
            ,'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate'
            ,'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
            ,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate'
            ,'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate'
            ,'dst_host_srv_rerror_rate','attack','outcome'])

testpl = pd.read_csv('NSL-KDD-Dataset/input/nslkdd/KDDTest+.txt',header = None,names=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent'
            ,'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root'
            ,'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login'
            ,'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate'
            ,'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
            ,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate'
            ,'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate'
            ,'dst_host_srv_rerror_rate','attack','outcome'])

valid = pd.read_csv('NSL-KDD-Dataset/input/nslkdd/KDDTest-21.txt',header = None,names=['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent'
            ,'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root'
            ,'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login'
            ,'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate'
            ,'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
            ,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate'
            ,'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate'
            ,'dst_host_srv_rerror_rate','attack','outcome'])

In [ ]:
#Preprocess to encode non numeric data
from sklearn.preprocessing import LabelEncoder

for i in ['protocol_type',"service","flag"]:
    encoder = LabelEncoder()
    fi = pd.concat([data[i],testpl[i]]).values
    encoder.fit(fi)
    data[i] = encoder.transform(data[i])
    testpl[i] = encoder.transform(testpl[i])

In [ ]:
#Relable the loaded data into 5 major classes
def lab(data):

    classlist = []

    check1 = ("apache2","back","land","neptune","mailbomb","pod","processtable","smurf","teardrop","udpstorm","worm")
    check2 = ("ipsweep","mscan","nmap","portsweep","saint","satan")
    check3 = ("buffer_overflow","loadmodule","perl","ps","rootkit","sqlattack","xterm")
    check4 = ("ftp_write","guess_passwd","httptunnel","imap","multihop","named","phf","sendmail","Snmpgetattack","spy","snmpguess","warezclient","warezmaster","xlock","xsnoop")


    for item in data.pop('attack'):
        if item in check1:
            classlist.append("DoS")
        elif item in check2:
            classlist.append("Probe")
        elif item in check3:
            classlist.append("U2R")
        elif item in check4:
            classlist.append("R2L")
        else:
            classlist.append("Normal")

    return classlist

In [ ]:
#seperating output vectors
y_train = np.array(lab(data))
y_test = np.array(lab(testpl))
y_valid = np.array(lab(valid))

In [ ]:
data.dtypes

duration                         int64
protocol_type                    int64
service                          int64
flag                             int64
src_bytes                        int64
dst_bytes                        int64
land                             int64
wrong_fragment                   int64
urgent                           int64
hot                              int64
num_failed_logins                int64
logged_in                        int64
num_compromised                  int64
root_shell                       int64
su_attempted                     int64
num_root                         int64
num_file_creations               int64
num_shells                       int64
num_access_files                 int64
num_outbound_cmds                int64
is_host_login                    int64
is_guest_login                   int64
count                            int64
srv_count                        int64
serror_rate                    float64
srv_serror_rate          

In [ ]:
#One-hot encoding
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
        df = df.drop(each, 1)
    return df
#Merging train and test data
combined_data = pd.concat([data,testpl])
#Applying one hot encoding to combined data
combined_data = one_hot(combined_data,['protocol_type',"service",'flag'])
#Function to min-max normalize
def normalize(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode
    @return a DataFrame with normalized specified features
    """
    result = df.copy() # do not touch the original df
    for feature_name in cols:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        if max_value > min_value:
            result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

#Normalizing training set
combined_data = normalize(combined_data,combined_data.columns[:-1])
combined_data.head()

<ipython-input-7-fe0b009bacc3>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
<ipython-input-7-fe0b009bacc3>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)
<ipython-input-7-fe0b009bacc3>:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(each, 1)


,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_1,flag_2,flag_3,flag_4,flag_5,flag_6,flag_7,flag_8,flag_9,flag_10
0,0.0,3.558064e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.0,1.057999e-07,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.0,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
3,0.0,1.681203e-07,6.223962e-06,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.0,1.442067e-07,3.206260e-07,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [ ]:
data = combined_data[:len(data)]
# data = combined_data.concat(axis=1).copy()
data['attack'] = y_train
testpl = combined_data[len(data):]
# testpl =  combined_data.concat(axis=1).copy()


<ipython-input-8-f222414c2405>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['attack'] = y_train
<ipython-input-8-f222414c2405>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['attack'] = y_train


In [ ]:
# maj_3 = data.loc[(data['attack']!='U2R') & (data['attack']!='R2L')]
maj_3 = data.loc[(data['attack']!= 'None')]

In [ ]:
!pip install tensorflow_addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 11.8 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# apply on df
encoder.fit(data['attack'].values.reshape(-1,1))
color_1hot = encoder.transform(maj_3['attack'].values.reshape(-1,1))

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#Customizing the MLP model
import tensorflow as tf
import tensorflow_addons as tfa

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units = 64, activation = 'relu', input_shape = (maj_3.shape[1]-1,)))
model.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
model.add(tf.keras.layers.Dense(units = 5, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [tf.keras.metrics.CategoricalAccuracy(),
                           tfa.metrics.F1Score(num_classes=5, average='macro'),
                           tfa.metrics.FBetaScore(beta=2.0, num_classes=5, average='macro')])


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
#Training the model on training data for 20 epochs with 80/20 training/validation split
tf.config.run_functions_eagerly(True)

model.fit(maj_3.drop(columns=['attack']).values, color_1hot, epochs = 20 ,validation_split=0.2)

Epoch 1/10


/usr/local/lib/python3.9/dist-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


3150/3150 [==============================] - 158s 49ms/step - loss: 0.0441 - categorical_accuracy: 0.9874 - f1_score: 0.7676 - fbeta_score: 0.7612 - val_loss: 0.0148 - val_categorical_accuracy: 0.9954 - val_f1_score: 0.8533 - val_fbeta_score: 0.8358
Epoch 2/10
3150/3150 [==============================] - 153s 49ms/step - loss: 0.0131 - categorical_accuracy: 0.9955 - f1_score: 0.9171 - fbeta_score: 0.9023 - val_loss: 0.0111 - val_categorical_accuracy: 0.9967 - val_f1_score: 0.8207 - val_fbeta_score: 0.8135
Epoch 3/10
3150/3150 [==============================] - 152s 48ms/step - loss: 0.0101 - categorical_accuracy: 0.9964 - f1_score: 0.9357 - fbeta_score: 0.9257 - val_loss: 0.0108 - val_categorical_accuracy: 0.9967 - val_f1_score: 0.8780 - val_fbeta_score: 0.8600
Epoch 4/10
3064/3150 [============================>.] - ETA: 3s - loss: 0.0085 - categorical_accuracy: 0.9967 - f1_score: 0.9429 - fbeta_score: 0.9320

In [ ]:
#Summarize precision, recall and F1 score for each classes
from sklearn.metrics import classification_report

print(classification_report(np.argmax(encoder.transform(y_test.reshape(-1,1)),axis=1),np.argmax(model.predict(testpl.values),axis=1)))

 10/705 [..............................] - ETA: 4s 

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


705/705 [==============================] - 8s 11ms/step
              precision    recall  f1-score   support

           0       0.95      0.85      0.90      7460
           1       0.71      0.96      0.81      9889
           2       0.80      0.63      0.70      2421
           3       0.70      0.09      0.16      2707
           4       0.26      0.67      0.37        67

    accuracy                           0.78     22544
   macro avg       0.68      0.64      0.59     22544
weighted avg       0.79      0.78      0.75     22544



In [ ]:

from imblearn.over_sampling import SMOTE, ADASYN
X_resampled, y_resampled = ADASYN(sampling_strategy = 'minority').fit_resample(data.drop(columns=['attack']).values,data['attack'].values)
X_resampled, y_resampled = ADASYN(sampling_strategy = 'minority').fit_resample(X_resampled, y_resampled)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0)
X_resampled, y_resampled = rus.fit_resample(data.drop(columns=['attack']).values,data['attack'].values)

In [ ]:
y_resampled = encoder.transform(y_resampled.reshape(-1,1))

In [ ]:
import sklearn
weights=sklearn.utils.class_weight.compute_class_weight(class_weight='balanced', classes=data['attack'].unique().tolist(), y=data['attack']).tolist()

In [ ]:
y_resampled.shape


(260, 5)

In [ ]:
model.fit(X_resampled, y_resampled,epochs=5,class_weight=dict(enumerate(weights)))

Epoch 1/5
9/9 [==============================] - 1s 112ms/step - loss: 0.1515 - categorical_accuracy: 0.9846 - f1_score: 0.9846 - fbeta_score: 0.9845
Epoch 2/5
9/9 [==============================] - 1s 104ms/step - loss: 0.1640 - categorical_accuracy: 0.9808 - f1_score: 0.9807 - fbeta_score: 0.9805
Epoch 3/5
9/9 [==============================] - 0s 46ms/step - loss: 0.1474 - categorical_accuracy: 0.9808 - f1_score: 0.9807 - fbeta_score: 0.9805
Epoch 4/5
9/9 [==============================] - 0s 42ms/step - loss: 0.1454 - categorical_accuracy: 0.9846 - f1_score: 0.9846 - fbeta_score: 0.9845
Epoch 5/5
9/9 [==============================] - 0s 44ms/step - loss: 0.1548 - categorical_accuracy: 0.9885 - f1_score: 0.9885 - fbeta_score: 0.9884


In [ ]:
print(classification_report(np.argmax(encoder.transform(y_test.reshape(-1,1)),axis=1),np.argmax(model.predict(testpl.values),axis=1)))

 31/705 [>.............................] - ETA: 2s

/usr/local/lib/python3.8/dist-packages/tensorflow/python/data/ops/structured_function.py:256: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


705/705 [==============================] - 3s 4ms/step
              precision    recall  f1-score   support

           0       0.93      0.86      0.90      7460
           1       0.81      0.92      0.86      9889
           2       0.78      0.73      0.76      2421
           3       0.67      0.41      0.51      2707
           4       0.10      0.69      0.17        67

    accuracy                           0.82     22544
   macro avg       0.66      0.72      0.64     22544
weighted avg       0.83      0.82      0.82     22544



In [ ]:
weights

[0.3741235169208381,
 0.5485792670977856,
 25.321206030150755,
 2.1615133836650653,
 484.51153846153846]

# Federated Optimization

# Utils

In [ ]:
# !pip install tensorflow_addons
# !pip install fl_implementation_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement fl_implementation_utils (from versions: none)
ERROR: No matching distribution found for fl_implementation_utils


In [ ]:
import numpy as np
import random
import os
import pickle
import pandas as pd

#from imutils import paths
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K


/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import time

In [ ]:
def create_clients(data_list, label_list, num_clients=5, initial='clients'):
    ''' return: a dictionary with keys clients' names and value as
                data shards - tuple of datas and label lists.
        args:
            data_list: a list of numpy arrays of training data
            label_list:a list of binarized labels for each data
            num_client: number of fedrated members (clients)
            initials: the clients'name prefix, e.g, clients_1

    '''

    #create a list of client names
    client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

    #randomize the data
    data = list(zip(data_list, label_list))
    random.shuffle(data)

    #shard data and place at each client
    size = len(data)//num_clients
    shards = [data[i:i + size] for i in range(0, size*num_clients, size)]

    #number of clients must equal number of shards
    assert(len(shards) == len(client_names))

    return {client_names[i] : shards[i] for i in range(len(client_names))}

In [ ]:
def batch_data(data_shard, bs=64):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)

### Model

In [ ]:
class LocalModel:
    @staticmethod
    def build(shape, classes):
        model = Sequential()
        # model.add(Dense(500,input_shape=(shape,)))
        # model.add(Activation("relu"))
        # model.add(Dense(300))
        # model.add(Activation("relu"))
        # model.add(Dense(200))
        # model.add(Activation("relu"))
        # model.add(Dense(classes))
        # model.add(Activation("sigmoid"))
        model.add(tf.keras.layers.Dense(units = 64, activation = 'relu', input_shape = (maj_3.shape[1]-1,)))
        model.add(tf.keras.layers.Dense(units = 64, activation = 'relu', input_shape = (maj_3.shape[1]-1,)))
        model.add(tf.keras.layers.Dense(units = 128, activation = 'relu'))
        model.add(tf.keras.layers.Dense(units = classes, activation = 'softmax'))
        model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = [tf.keras.metrics.CategoricalAccuracy(),
                           tfa.metrics.F1Score(num_classes=classes, average='macro'),
                           tfa.metrics.FBetaScore(beta=2.0, num_classes=classes, average='macro')])
        return model

### Weight calculation util

In [ ]:
def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count

In [ ]:
def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final

In [ ]:
def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)

    return avg_grad

In [ ]:
def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
    #logits = model.predict(X_test, batch_size=100)
    logits = model.predict(X_test)
    #print(logits)
    loss = cce(Y_test, logits)
    acc = accuracy_score( tf.argmax(Y_test, axis=1),tf.argmax(logits, axis=1))
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {}'.format(comm_round, acc, loss))
    return acc, loss

# Create Clients

In [ ]:
#create clients
clients = create_clients(maj_3.drop(columns=['attack']).values, color_1hot, num_clients=5, initial='client')

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

In [ ]:
#process and batch the test set
# test_batched = tf.data.Dataset.from_tensor_slices(testpl.values, encoder.transform(y_test.reshape(-1,1)).all()).batch(len(y_test))
test_batched = tf.data.Dataset.from_tensor_slices((testpl.values, encoder.transform(y_test.reshape(-1,1)))).batch(len(y_test))

In [ ]:
comms_round = 5 #number of global epochs

# #create optimizer
# lr = 0.01
# loss='categorical_crossentropy'
# metrics = ['accuracy']
# optimizer = SGD(lr=lr,
#                 decay=lr / comms_round,
#                 momentum=0.9
#                )

#initialize global model
#print(data_list.shape,labels)
smlp_global = LocalModel()
global_model = smlp_global.build(maj_3.shape[1],5)


In [ ]:
print(test_batched)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 123), dtype=tf.float64, name=None), TensorSpec(shape=(None, 5), dtype=tf.float64, name=None))>


In [ ]:
#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    for client in tqdm(client_names , desc = 'Progress Bar'):
        #time.sleep(0.5)
        smlp_local = LocalModel()
        local_model = smlp_local.build(maj_3.shape[1],5)
        # local_model.compile(loss=loss,
        #               optimizer=optimizer,
        #               metrics=metrics)

        #print(local_model.summary())
        #print(clients_batched)
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=30, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
       print(classification_report(np.argmax(Y_test,axis=1),np.argmax(global_model.predict(X_test),axis=1)))
      # global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

Progress Bar: 100%|██████████| 5/5 [03:51<00:00, 46.28s/it]


705/705 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.87      0.82      0.84      7460
           1       0.70      0.91      0.79      9889
           2       0.80      0.73      0.76      2421
           3       0.81      0.14      0.24      2707
           4       0.71      0.07      0.14        67

    accuracy                           0.77     22544
   macro avg       0.78      0.54      0.55     22544
weighted avg       0.78      0.77      0.74     22544



Progress Bar: 100%|██████████| 5/5 [03:33<00:00, 42.61s/it]

  1/705 [..............................] - ETA: 22s

705/705 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      7460
           1       0.75      0.91      0.82      9889
           2       0.73      0.72      0.72      2421
           3       0.83      0.28      0.42      2707
           4       0.20      0.42      0.27        67

    accuracy                           0.79     22544
   macro avg       0.68      0.63      0.62     22544
weighted avg       0.80      0.79      0.77     22544



...

  1/705 [..............................] - ETA: 22s

705/705 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.92      0.84      0.88      7460
           1       0.73      0.91      0.81      9889
           2       0.68      0.73      0.70      2421
           3       0.88      0.22      0.36      2707
           4       0.20      0.55      0.30        67

    accuracy                           0.78     22544
   macro avg       0.68      0.65      0.61     22544
weighted avg       0.80      0.78      0.77     22544



Progress Bar: 100%|██████████| 5/5 [03:52<00:00, 46.53s/it]

  1/705 [..............................] - ETA: 21s

705/705 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.88      0.84      0.86      7460
           1       0.73      0.91      0.81      9889
           2       0.74      0.73      0.73      2421
           3       0.94      0.21      0.34      2707
           4       0.23      0.63      0.34        67

    accuracy                           0.78     22544
   macro avg       0.70      0.66      0.62     22544
weighted avg       0.81      0.78      0.76     22544



In [ ]:
# def installRequirementOnPath(path):
#   !pip install -qr $path

In [ ]:
# def create_clients(data_list, num_clients=3, initial='clients'):
#     ''' return: a dictionary with keys clients' names and value as
#                 data shards - tuple of datas and label lists.
#         args:
#             data_list: a list of numpy arrays of training data
#             label_list:a list of binarized labels for each data
#             num_client: number of fedrated members (clients)
#             initials: the clients'name prefix, e.g, clients_1

#     '''

#     #create a list of client names
#     client_names = ['{}_{}'.format(initial, i+1) for i in range(num_clients)]

#     #shard data and place at each client
#     shards = np.array_split(maj_3, num_clients)

#     #number of clients must equal number of shards
#     assert(len(shards) == len(client_names))

#     return {client_names[i] : pd.DataFrame(shards[i]) for i in range(len(client_names))}


# Oversampling

In [ ]:
import pandas as pd
from imblearn.over_sampling import ADASYN

# Load the NSL-KDD dataset into a pandas DataFrame
df = maj_3
print(df.iloc[:, -1].value_counts())

# Separate the features and target variable
# X = df.iloc[:, :-1]
X = df.drop(columns=['attack'])
y = df['attack'].values.reshape(-1,1)

# Oversample the minority class using ADASYN
ada = ADASYN()
X_resampled, y_resampled = ada.fit_resample(X, y)
# X_resampled2, y_resampled2 = ada.fit_resample(X_resampled, y_resampled)
# Create a new balanced DataFrame
df_resampled = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)

# Check the distribution of classes in the new DataFrame
print(df_resampled.iloc[:, -1].value_counts())


Normal    67343
DoS       45927
Probe     11656
R2L         995
U2R          52
Name: attack, dtype: int64
Probe     67391
R2L       67347
DoS       67344
Normal    67343
U2R       67333
Name: 0, dtype: int64


In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
# apply on df
encoder.fit((df_resampled.iloc[:, -1]).values.reshape(-1,1))
color_1hot_resampled = encoder.transform((df_resampled.iloc[:, -1]).values.reshape(-1,1))

/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
#create clients
clients = create_clients((df_resampled.iloc[: , :-1]).values, color_1hot_resampled, num_clients=5, initial='client')

In [ ]:
#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)

In [ ]:
#process and batch the test set
# test_batched = tf.data.Dataset.from_tensor_slices(testpl.values, encoder.transform(y_test.reshape(-1,1)).all()).batch(len(y_test))
test_batched = tf.data.Dataset.from_tensor_slices((testpl.values, encoder.transform(y_test.reshape(-1,1)))).batch(len(y_test))

In [ ]:
comms_round = 5 #number of global epochs

# #create optimizer
# lr = 0.01
# loss='categorical_crossentropy'
# metrics = ['accuracy']
# optimizer = SGD(lr=lr,
#                 decay=lr / comms_round,
#                 momentum=0.9
#                )

#initialize global model
#print(data_list.shape,labels)
smlp_global = LocalModel()
global_model = smlp_global.build(df_resampled.shape[1],5)

In [ ]:
#commence global training loop
for comm_round in range(comms_round):

    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()

    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)

    #loop through each client and create new local model
    for client in tqdm(client_names , desc = 'Progress Bar'):
        #time.sleep(0.5)
        smlp_local = LocalModel()
        local_model = smlp_local.build(df_resampled.shape[1],5)
        # local_model.compile(loss=loss,
        #               optimizer=optimizer,
        #               metrics=metrics)

        #print(local_model.summary())
        #print(clients_batched)
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)

        #fit local model with client's data
        local_model.fit(clients_batched[client], epochs=25, verbose=0)

        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)

        #clear session to free memory after each communication round
        K.clear_session()

    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)

    #update global model
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
       print(classification_report(np.argmax(Y_test,axis=1),np.argmax(global_model.predict(X_test),axis=1)))
      # global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)

Progress Bar: 100%|██████████| 5/5 [08:35<00:00, 103.11s/it]


705/705 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.92      0.86      0.89      7460
           1       0.80      0.90      0.85      9889
           2       0.77      0.82      0.79      2421
           3       0.93      0.41      0.57      2707
           4       0.08      0.69      0.15        67

    accuracy                           0.82     22544
   macro avg       0.70      0.74      0.65     22544
weighted avg       0.85      0.82      0.82     22544



Progress Bar: 100%|██████████| 5/5 [08:39<00:00, 103.82s/it]

 35/705 [>.............................] - ETA: 1s 

705/705 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      7460
           1       0.80      0.91      0.85      9889
           2       0.78      0.80      0.79      2421
           3       0.83      0.34      0.48      2707
           4       0.11      0.67      0.19        67

    accuracy                           0.81     22544
   macro avg       0.68      0.72      0.64     22544
weighted avg       0.83      0.81      0.81     22544



...

Progress Bar: 100%|██████████| 5/5 [08:48<00:00, 105.72s/it]

  1/705 [..............................] - ETA: 21s

705/705 [==============================] - 1s 1ms/step
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      7460
           1       0.76      0.91      0.83      9889
           2       0.74      0.73      0.73      2421
           3       0.78      0.25      0.38      2707
           4       0.13      0.61      0.21        67

    accuracy                           0.79     22544
   macro avg       0.66      0.67      0.61     22544
weighted avg       0.81      0.79      0.78     22544



Progress Bar: 100%|██████████| 5/5 [08:40<00:00, 104.12s/it]


705/705 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.90      0.86      0.88      7460
           1       0.76      0.91      0.83      9889
           2       0.74      0.72      0.73      2421
           3       0.76      0.23      0.35      2707
           4       0.11      0.63      0.19        67

    accuracy                           0.79     22544
   macro avg       0.66      0.67      0.60     22544
weighted avg       0.80      0.79      0.78     22544

